In [1]:
import torch
import numpy as np

# shape 변경하기

In [2]:
A = np.ones((100, 120, 3))

In [3]:
A_transposed = np.transpose(A, (2, 0, 1))
print(A_transposed.shape)

(3, 100, 120)


차원 인덱스를 이용하여 맘대로 크기 변경가능

In [4]:
B = torch.tensor(A)
B_transposed = torch.transpose(B, 1, 2)
print(B_transposed.shape)

torch.Size([100, 3, 120])


In [5]:
B_transposed2 = torch.transpose(B_transposed, 0, 1)
print(B_transposed2.shape)

torch.Size([3, 100, 120])


torch의 경우 한번에 2개의 채널만을 교환 가능하다.

따라서 numpy로 바꾸는 것이 편리하다

# permute를 이용한 shape 변경

In [6]:
A = torch.ones(1, 3, 10, 20)
print(A.shape)

torch.Size([1, 3, 10, 20])


In [7]:
A_permute = A.permute(1, 3, 0, 2)
print(A_permute.shape)

torch.Size([3, 20, 1, 10])


numpy.transpose 처럼 한번에 채널 변경 가능

# nn.Dropout vs F.dropout

Dropout의 경우 일정 비율만큼 텐서값을 0으로 만든다.

In [8]:
import torch.nn as nn

In [9]:
class Model1(nn.Module):
    def __init__(self, p = 0.0): # F.dropout
        super().__init__()
        self.p = p
    
    def forward(self, inputs):
        return nn.functional.dropout(inputs, p = self.p, training=True)
    
class Model2(nn.Module):
    def __init__(self, p = 0.0):
        super().__init__()
        self.drop_layer = nn.Dropout(p=p)
    
    def forward(self, inputs):
        return self.drop_layer(inputs)

In [10]:
model1 = Model1(p = 0.5)
model2 = Model2(p = 0.5)

In [11]:
inputs = torch.rand(10)

print('Normal (train) model:')
print('Model 1:', model1(inputs))
print('Model 2:', model2(inputs))

Normal (train) model:
Model 1: tensor([1.3858, 0.0000, 0.0000, 0.0000, 0.0000, 0.6708, 1.8110, 1.4070, 0.0000,
        0.0000])
Model 2: tensor([1.3858, 0.9440, 0.0000, 0.0000, 0.6535, 0.6708, 0.0000, 0.0000, 0.0000,
        1.4730])


즉 p = 0.5 이므로 50%의 확률로 0이 된다.

In [12]:
model1.eval()

Model1()

In [13]:
model2.eval()

Model2(
  (drop_layer): Dropout(p=0.5, inplace=False)
)

In [14]:
print('Evaluation mode:')
print('Model 1:', model1(inputs))
print('Model 2:', model2(inputs)) # 레이어가 활성화 안되었

Evaluation mode:
Model 1: tensor([1.3858, 0.0000, 0.9405, 1.3051, 0.0000, 0.6708, 0.0000, 0.0000, 1.7718,
        0.0000])
Model 2: tensor([0.6929, 0.4720, 0.4702, 0.6526, 0.3267, 0.3354, 0.9055, 0.7035, 0.8859,
        0.7365])


In [15]:
print(model1)

Model1()


In [16]:
print(model2)

Model2(
  (drop_layer): Dropout(p=0.5, inplace=False)
)


드롭아웃의 경우 훈련중에만 설정하도록 평가를 하여야 하므로, 예측이나 모델을 평가할때에는 사용을 해제하여야 한다. 

F.dropout의 경우 평가 / 예측 모드에 따라 구분이 되지만, nn.dropout의 경우 평가 / 예측 모드에 따라 레이어가 활성화 다르다.

따라서 대부분 nn.dropout을 더 선호한다.

# nn.AvgPool2d vs nn.AdaptiveAvgPool2d

Pooling layer의 경우 CNN 연산후 값들을 뽑아내는데 이용된다.

AvgPooling의 경우 pooling 작업에 대한 kernel의 크기, stride 크기를 정의하여야 한다.

AdaptiveAvgPool의 경우 pooling 작업이 끝났을때 필요한 출력의 크기를 정의한다.

## nn.AvgPool2d

In [17]:
# 3x3 커널로 pooling
m = nn.AvgPool2d(3, stride=2)
# 비정사각 커널로 풀링
m = nn.AvgPool2d((3, 2), stride=(2, 1))
input = torch.randn(20, 16, 50, 32)

In [18]:
output = m(input)
print(output.shape)

torch.Size([20, 16, 24, 31])


## nn.AdaptiveAvgPool

In [19]:
m = nn.AdaptiveAvgPool2d((5, 7))
input = torch.randn(1, 64, 8, 9)
output = m(input)
print(output.shape)

torch.Size([1, 64, 5, 7])


2차원 텐서 크기가 5x7로 Pooling 됨

In [20]:
m = nn.AdaptiveAvgPool2d(7)
input = torch.randn(1, 64, 10, 9)
output = m(input)
print(output.shape)

torch.Size([1, 64, 7, 7])


Pooling 된 2차원 텐서 크기가 7x7임

In [21]:
m = nn.AdaptiveMaxPool2d((None, 7))
input = torch.randn(1, 64, 10, 9)
output = m(input)
print(output.shape)

torch.Size([1, 64, 10, 7])


Pooling 된 2차원 텐서의 크기가 widh 차원의 경우 유지됨

In [22]:
input = torch.tensor([[[[1,2.,3], [4,5,6], [7,8,9]]]], dtype = torch.float)
input

tensor([[[[1., 2., 3.],
          [4., 5., 6.],
          [7., 8., 9.]]]])

In [23]:
output = nn.AdaptiveAvgPool2d((2,2))(input)
output

tensor([[[[3., 4.],
          [6., 7.]]]])

3은 1, 2, 4, 5끼리

4는 2, 3, 5, 6끼리

6은 4, 5, 7, 8끼리

7은 5, 6, 7, 9끼리

# torch.einsum 사용하기

물리학에서 행렬계산을 직관적으로 하기 위하여 아인슈타인 표기를 많이 사용한다.

이러한 표기법이 토치에서 구현되어 있다.

In [24]:
x = torch.rand((2, 3))
x

tensor([[0.3435, 0.0892, 0.0737],
        [0.6109, 0.8787, 0.8669]])

In [25]:
torch.einsum('ij->', x)
# 전체 다 더하기

tensor(2.8628)

In [26]:
torch.einsum('ij->j', x)
# column 합

tensor([0.9544, 0.9679, 0.9406])

In [27]:
torch.einsum('ij->i', x)
# row 합

tensor([0.5064, 2.3564])

In [28]:
y = torch.rand((1, 3))
y

tensor([[0.7557, 0.1286, 0.4445]])

In [29]:
torch.einsum('ij,kj->ik', x, x)
# 2x3 과 3x2 가 곱해지므로 2x2가 나와야 함

tensor([[0.1314, 0.3521],
        [0.3521, 1.8967]])

In [30]:
torch.einsum('ij,kj->ik', x, y)
# 행렬곱

tensor([[0.3038],
        [0.9600]])

In [31]:
torch.einsum('ij,ij->', x, x)
# dot product

tensor(2.0281)

In [32]:
torch.einsum('i,i->', x[0], x[0])
# 첫째 행끼리 곱셈

tensor(0.1314)

In [33]:
torch.einsum('ij, ij->ij', x, x)

# Element wise product
# 각 행렬의 원소끼리 곱한 값

tensor([[0.1180, 0.0079, 0.0054],
        [0.3732, 0.7721, 0.7515]])

In [34]:
a = torch.rand((3))
b = torch.rand((5))
torch.einsum('i,j->ij', a, b)
# outer product

tensor([[0.2646, 0.3407, 0.2944, 0.3683, 0.3507],
        [0.1862, 0.2397, 0.2071, 0.2592, 0.2468],
        [0.1625, 0.2092, 0.1808, 0.2262, 0.2154]])

In [35]:
a = torch.rand((3, 2, 5))
b = torch.rand((3, 5, 3))
torch.einsum('ijk,ikl->ijl', a, b)

tensor([[[1.0732, 0.8128, 1.3856],
         [1.9057, 1.1245, 1.9013]],

        [[1.3025, 0.9771, 0.8946],
         [1.9841, 1.5569, 1.5701]],

        [[2.2415, 1.7755, 2.3344],
         [1.8472, 1.3612, 1.9189]]])

In [36]:
x = torch.rand((3, 3))
x

tensor([[0.2543, 0.8844, 0.8535],
        [0.5049, 0.2714, 0.2263],
        [0.8747, 0.1499, 0.9536]])

In [37]:
torch.einsum('ii->i', x)
# 대각요소만 추출

tensor([0.2543, 0.2714, 0.9536])

In [38]:
torch.einsum('ii->', x)
# 대각합

tensor(1.4793)

# torch.softmax

In [39]:
print(torch.softmax(torch.ones((1, 2, 3, 4)), dim=0))

# 값들끼리 비교해서 제일 높은 것을 반환. 값들이 전부 같으므로 1

tensor([[[[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]],

         [[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]]]])


In [40]:
print(torch.softmax(torch.ones((1, 2, 3, 4)), dim=1))

# 채널차원에서 비교. 전부 같으므로 0.5가 나옴

tensor([[[[0.5000, 0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000, 0.5000]],

         [[0.5000, 0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000, 0.5000]]]])


In [41]:
print(torch.softmax(torch.ones((1, 2, 3, 4)), dim=2))

# 행 끼리 비교. 3개가 다 같으므로 1/3

tensor([[[[0.3333, 0.3333, 0.3333, 0.3333],
          [0.3333, 0.3333, 0.3333, 0.3333],
          [0.3333, 0.3333, 0.3333, 0.3333]],

         [[0.3333, 0.3333, 0.3333, 0.3333],
          [0.3333, 0.3333, 0.3333, 0.3333],
          [0.3333, 0.3333, 0.3333, 0.3333]]]])


In [42]:
print(torch.softmax(torch.ones((1, 2, 3, 4)), dim=3))

# 열끼리 비교. 4개 다 같으므로 1/4

tensor([[[[0.2500, 0.2500, 0.2500, 0.2500],
          [0.2500, 0.2500, 0.2500, 0.2500],
          [0.2500, 0.2500, 0.2500, 0.2500]],

         [[0.2500, 0.2500, 0.2500, 0.2500],
          [0.2500, 0.2500, 0.2500, 0.2500],
          [0.2500, 0.2500, 0.2500, 0.2500]]]])


In [43]:
print(torch.softmax(torch.ones((1, 2, 3, 4)), dim=-1))

# 제일 높은 차원으로 자동 계산

tensor([[[[0.2500, 0.2500, 0.2500, 0.2500],
          [0.2500, 0.2500, 0.2500, 0.2500],
          [0.2500, 0.2500, 0.2500, 0.2500]],

         [[0.2500, 0.2500, 0.2500, 0.2500],
          [0.2500, 0.2500, 0.2500, 0.2500],
          [0.2500, 0.2500, 0.2500, 0.2500]]]])


# torch.repeat 사용하기

In [44]:
torch.ones(1)

tensor([1.])

In [45]:
torch.ones(1).repeat(2, 3)

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [46]:
torch.ones(2, 3)

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [47]:
torch.ones(2, 3).repeat(3, 4).shape

torch.Size([6, 12])

In [48]:
torch.ones(2, 3).repeat(3, 4, 5).shape

torch.Size([3, 8, 15])

In [49]:
torch.arange(0, 4)

tensor([0, 1, 2, 3])

In [50]:
torch.arange(0, 4).repeat(3, 4)

tensor([[0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3],
        [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3],
        [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]])

# torch.scatter

특정 인덱스에 특정 값을 뿌리는 특징을 가지고 있습니다.

In [51]:
src = torch.arange(1, 11).reshape((2, 5))
print(src)

tensor([[ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10]])


In [52]:
index = torch.tensor([[0, 1, 2, 0, 2]])
torch.zeros(3, 5, dtype = src.dtype).scatter_(0, index, src)

tensor([[1, 0, 0, 4, 0],
        [0, 2, 0, 0, 0],
        [0, 0, 3, 0, 5]])

뭐지 싶을 건데, dim = 0 이므로 값을 기준으로 scatter 진행.

src에서 값을 가져다가 원래 토치에 흩뿌린다 생각하면 된다.

인덱스 텐서의 크기가 5이므로 src 텐서에서 5개의 텐서만 박힌다.

인덱스가 0, 1, 2, 0, 2 이므로 순서대로 0번째, 1번째, 2번째, 0번째, 2번째에

1 2 3 4 5 가 박힐 것이다.

In [53]:
src = torch.arange(1, 11).reshape((2, 5))
print(src)

tensor([[ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10]])


In [54]:
index = torch.tensor([[0, 1, 2], [0, 1, 4]])
torch.zeros(3, 5, dtype=src.dtype).scatter_(1, index, src)

tensor([[1, 2, 3, 0, 0],
        [6, 7, 0, 0, 8],
        [0, 0, 0, 0, 0]])

여기서는 dim이 1이므로, 줄 단위로 생각을 할 것이다.

첫번째 줄은 0, 1, 2 이므로 1, 2, 3이

두번째 줄은 0, 1, 4 이므로 6, 7, 8이 순서대로 각각의 zeros 텐서에 넣어질 것이다.

# torch.split

In [55]:
a = torch.arange(10).reshape(5,2)
a

tensor([[0, 1],
        [2, 3],
        [4, 5],
        [6, 7],
        [8, 9]])

In [56]:
torch.split(a, 2)

# 2개씩 분해

(tensor([[0, 1],
         [2, 3]]),
 tensor([[4, 5],
         [6, 7]]),
 tensor([[8, 9]]))

In [57]:
torch.split(a, [1,4])

(tensor([[0, 1]]),
 tensor([[2, 3],
         [4, 5],
         [6, 7],
         [8, 9]]))

# torch.nan_to_num

In [58]:
x = torch.tensor([float('nan'), float('inf'), -float('inf'), 3.14])
torch.nan_to_num(x)

tensor([ 0.0000e+00,  3.4028e+38, -3.4028e+38,  3.1400e+00])

In [59]:
torch.nan_to_num(x, nan = 0.0, posinf=1e10, neginf=-1e10)

tensor([ 0.0000e+00,  1.0000e+10, -1.0000e+10,  3.1400e+00])

# weight 초기화 방법

In [60]:
import torch.nn.functional as F 

In [61]:
class CNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=6,
            kernel_size=(3,3),
            stride=(1,1),
            padding=(1,1)
        )
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(
            in_channels=6,
            out_channels=16,
            kernel_size=(3,3),
            stride=(1,1),
            padding=(1,1)
        )
        
        self.fc1 = nn.Linear(16*7*7, num_classes)
        
        self.initialize_weights()
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        
        return x
    
    def initialize_weights(self):
        for m in self.modules():
            # convolution kernel의 가중치를 He initialization
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_uniform_(m.weight)
                
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
                    
                elif isinstance(m, nn.BatchNorm2d):
                    nn.init.constant_(m.weight, 1)
                    nn.init.constant_(m.bias, 0)
        
                elif isinstance(m, nn.Linear):
                    nn.init.kaiming_uniform_(m.weight)
                    nn.init.constant_(m.bias, 0)
                    
if __name__ == '__main__':
    model = CNN(in_channels=3,num_classes=10)
    
    # He initialization과 Constant로 초기화 한것을 확인할 수 있습니다.
    for param in model.parameters():
        print(param)

Parameter containing:
tensor([[[[ 0.1103, -0.3843,  0.2854],
          [ 0.1306,  0.2139,  0.0752],
          [ 0.1389,  0.2022,  0.4347]],

         [[ 0.2956,  0.0851, -0.4342],
          [-0.1576,  0.0331,  0.0029],
          [-0.1181,  0.4160, -0.3398]],

         [[ 0.0033, -0.1967,  0.2678],
          [-0.2506, -0.1710,  0.0164],
          [ 0.3866, -0.2206,  0.3805]]],


        [[[ 0.3844,  0.0979,  0.3595],
          [-0.2392, -0.2969,  0.2484],
          [-0.3168,  0.2796, -0.1044]],

         [[-0.2922, -0.2327,  0.2301],
          [-0.4593,  0.2892,  0.3827],
          [-0.2698,  0.0184, -0.1722]],

         [[-0.3666,  0.2382, -0.2350],
          [-0.0984, -0.1140, -0.3218],
          [-0.0890, -0.2260,  0.0523]]],


        [[[ 0.1532,  0.0992, -0.0092],
          [ 0.4230,  0.3932, -0.1153],
          [-0.3190,  0.2027,  0.0891]],

         [[-0.2769,  0.4501, -0.2236],
          [ 0.2910, -0.1907, -0.1876],
          [ 0.2373,  0.2757,  0.3940]],

         [[ 0.1737, -0

# DataLoader 활용

In [62]:
from torchvision import datasets, transforms

In [63]:
batch_size = 32
test_batch_size = 32

In [66]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        root='datasets/', # 현재 위치에 저장
        train = True, # 학습용 데이터
        download=True, # 다운로드
        transform=transforms.Compose([
                transforms.ToTensor(), # 데이터를 tensor 타입으로 변경
                transforms.Normalize(mean=(0.5,), std=(0.5,)) # 정규화 진행
        ])
    ),
    batch_size=batch_size,
    shuffle=True
) # 학습세트

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting datasets/MNIST\raw\train-images-idx3-ubyte.gz to datasets/MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting datasets/MNIST\raw\train-labels-idx1-ubyte.gz to datasets/MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting datasets/MNIST\raw\t10k-images-idx3-ubyte.gz to datasets/MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting datasets/MNIST\raw\t10k-labels-idx1-ubyte.gz to datasets/MNIST\raw



In [67]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        root = "datasets/", # 현재 경로에 datasets/MNIST/ 를 생성 후 데이터를 저장한다.
        train = False, # test 용도의 data 셋을 저장한다.
        download = True,
        transform = transforms.Compose([
            transforms.ToTensor(), # tensor 타입으로 데이터 변경
            transforms.Normalize(mean = (0.5,), std = (0.5,)) # train과 동일한 조건으로 normalize 한다.            
        ])
    ),
    batch_size = test_batch_size,
    shuffle = True
)

In [68]:
image, label = next(iter(train_loader))
# iter로 반복하는 객체를 설정해주고, next로 반복할때마다 다음 구성요소를 불러냄
print(image.shape)
# torch.Size([32, 1, 28, 28])

print(label.shape)
# torch.Size([32])

torch.Size([32, 1, 28, 28])
torch.Size([32])
